In [1]:
import pandas as pd
import numpy as np
import joblib
import zipfile
import os

### Loading data

In [2]:
# Load the model
consumption = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/train.csv') # Replace with "pickle.load" if you used pickle
building = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/building_metadata.csv')
weather = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/weather_train.csv')


In [3]:
len(consumption[consumption['meter_reading']<= 0])/len(consumption[consumption['meter'] == 0])

0.1553760039665332

In [4]:
consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [5]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.1+ KB


In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


### Filtering consumption only for electrictiy readings

In [28]:
electricity = consumption[(consumption['meter'] == 0)]

In [29]:
electricity = electricity.drop(columns='meter')

In [30]:
electricity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12060910 entries, 0 to 20216099
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 368.1+ MB


In [31]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


In [32]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


### Merging everything into two datasets

In [63]:
electricity_building = electricity.merge(building, on='building_id', how='left')


electricity_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
 3   site_id        int64  
 4   primary_use    object 
 5   square_feet    int64  
 6   year_built     float64
 7   floor_count    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 736.1+ MB


In [64]:
electricity_building_weather = electricity_building.merge(weather, on=['site_id', 'timestamp'], how='left')


electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   timestamp           object 
 2   meter_reading       float64
 3   site_id             int64  
 4   primary_use         object 
 5   square_feet         int64  
 6   year_built          float64
 7   floor_count         float64
 8   air_temperature     float64
 9   cloud_coverage      float64
 10  dew_temperature     float64
 11  precip_depth_1_hr   float64
 12  sea_level_pressure  float64
 13  wind_direction      float64
 14  wind_speed          float64
dtypes: float64(10), int64(3), object(2)
memory usage: 1.3+ GB


### Drop buildings where year_built >= 2016

In [78]:
electricity_building_weather = electricity_building_weather[(electricity_building_weather['year_built'] < 2016) | (electricity_building_weather['year_built'].isna()) ]

### Analyze data

In [80]:
electricity_building_weather[electricity_building_weather['meter_reading']==0].site_id.unique()

array([ 0,  5,  7,  8,  9, 13, 15,  3,  4, 12,  2, 14, 10,  6,  1])

In [102]:
electricity_building_weather.groupby('site_id').timestamp.count()

site_id
0      864557
1      447982
2     1184512
3     2352533
4      743281
5      781776
6      315436
7      104784
8      567915
9     1068748
10     236573
11      43400
12     315909
13    1236660
14     890768
15     841195
Name: timestamp, dtype: int64

### Saving merged datasets

In [82]:
# Specify the zip file name
zip_filename = "../dataset/filtered.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "filtered.pkl"
    joblib.dump(electricity_building_weather, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="filtered.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)


## Filter hours for 08.00 - 18.00

In [87]:
electricity_building_weather_for_models = electricity_building_weather[(pd.to_datetime(electricity_building_weather['timestamp']).dt.hour >= 8) & (pd.to_datetime(electricity_building_weather['timestamp']).dt.hour <= 18)]

In [88]:
electricity_building_weather_for_models['meter_reading'].describe()

count    5.496696e+06
mean     1.884246e+02
std      4.170514e+02
min      0.000000e+00
25%      2.200000e+01
50%      7.300000e+01
75%      1.884000e+02
max      4.433650e+04
Name: meter_reading, dtype: float64

In [90]:
electricity_building_weather_for_models[electricity_building_weather_for_models['site_id']==6].meter_reading.describe()

count    144264.000000
mean        192.170347
std         239.269865
min           0.000000
25%          46.833100
50%         101.075000
75%         215.067000
max        1296.940000
Name: meter_reading, dtype: float64

In [93]:
electricity_building_weather_for_models = electricity_building_weather_for_models[electricity_building_weather_for_models['site_id'].isin([1, 6])]

In [94]:
electricity_building_weather_for_models.describe()

,building_id,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,349588.000000,349588.000000,349588.000000,349588.000000,161038.000000,205324.000000,348586.000000,122843.000000,348586.000000,144228.000000,344163.000000,333325.000000,348277.000000
mean,392.826041,185.070463,3.063343,76440.721072,1961.474894,7.000049,13.799384,0.514258,7.550489,0.806702,1016.743762,169.882097,3.553334
std,313.859668,217.393339,2.461574,59601.165754,31.395331,2.504867,8.107633,1.547535,8.134555,5.622529,9.434888,111.640308,2.417984
min,105.000000,0.000000,1.000000,5374.000000,1900.000000,2.000000,-13.300000,0.000000,-22.800000,-1.000000,978.700000,0.000000,0.000000
25%,126.000000,45.187750,1.000000,39558.000000,1953.500000,5.000000,7.900000,0.000000,2.200000,0.000000,1011.900000,60.000000,2.100000
50%,148.000000,99.592600,1.000000,62893.000000,1960.000000,7.000000,13.300000,0.000000,8.200000,0.000000,1017.700000,200.000000,3.100000
75%,764.000000,231.200000,6.000000,100481.000000,1990.500000,8.000000,19.600000,0.000000,13.300000,0.000000,1022.600000,260.000000,5.100000
max,788.000000,1438.500000,6.000000,401753.000000,2007.000000,16.000000,36.700000,9.000000,24.400000,89.000000,1045.200000,360.000000,16.500000


In [105]:
electricity_building_weather_for_models.groupby(['site_id','primary_use']).timestamp.count()

site_id  primary_use                  
1        Education                        88572
         Entertainment/public assembly     4026
         Lodging/residential              40260
         Office                           64414
         Public services                   8052
6        Education                        52070
         Entertainment/public assembly    12055
         Lodging/residential              44190
         Office                           31930
         Public services                   4019
Name: timestamp, dtype: int64

In [98]:
electricity_building_weather_for_models[electricity_building_weather_for_models['primary_use'] == 'Education'].meter_reading.describe()

count    140642.000000
mean        302.611111
std         281.331626
min           0.000000
25%          73.069525
50%         199.700000
75%         473.000000
max        1438.500000
Name: meter_reading, dtype: float64

In [106]:
electricity_building_weather_for_models[(electricity_building_weather_for_models['primary_use'] == 'Education') & (electricity_building_weather_for_models['site_id'] == 1)].meter_reading.describe()

count    88572.000000
mean       292.208059
std        238.790117
min          0.000000
25%        115.300000
50%        231.355000
75%        425.500000
max       1438.500000
Name: meter_reading, dtype: float64

In [107]:
electricity_building_weather_for_models[(electricity_building_weather_for_models['primary_use'] == 'Education') & (electricity_building_weather_for_models['site_id'] == 6)].meter_reading.describe()

count    52070.000000
mean       320.306888
std        341.013316
min          0.000000
25%         50.951450
50%        122.025000
75%        520.708500
max       1296.940000
Name: meter_reading, dtype: float64

In [108]:
# Specify the zip file name
zip_filename = "../dataset/filtered_for_models.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "filtered_for_models.pkl"
    joblib.dump(electricity_building_weather_for_models, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="filtered_for_models.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)